# Leveraging the Word vec Result to clustering
### Classify industries according to BERT result

In [9]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [10]:
file=eval(open('company dic i.txt').read())
comp_corr=dict(file)

In [11]:
company_index=dict()
max_num=len(comp_corr)
cov_matrix=np.matrix(np.zeros((1,max_num)))
count=0
for index in comp_corr:
    company_index[count]=index
    count+=1
    cov_matrix=np.vstack((cov_matrix,np.matrix(list(comp_corr[index].values()))))
cov_matrix=np.delete(cov_matrix,0,axis=0)

In [12]:
#query all company in the same industry
def query_ind(index):
    if type(company_index[index])==type('a'):
        yield index
    else:
        sub_index=company_index[index]
        for i in sub_index:
            for j in query_ind(i):
                yield j

# sum correlation
def sum_corr_func(index1,index2):
    return cov_matrix_enlarge[index1,index2]

# set the corr of companies within the same industry to 1
def elim_corr(comp_list):
    for i in comp_list:
        cov_matrix_enlarge[i,:]=0
        cov_matrix_enlarge[:,i]=0

In [30]:
cov_matrix_enlarge=np.copy(cov_matrix)
set_num_counter=list(range(max_num))
cur_num=max_num
exempt_set=set()
record_dummy = -1
while len(set(set_num_counter))>25:
    # get minimun correlation in the matrix
    min_pair=cov_matrix_enlarge.argmax()
    row=min_pair//cov_matrix_enlarge.shape[0]
    col=min_pair%cov_matrix_enlarge.shape[0]
    company_index[cur_num]=[row,col]
    if row == col:
        cov_matrix_enlarge[row, col] = 0
        continue
    cur_num+=1
    # calculate the new generated industry correlation with other companies
    ind_comp_list=[i for i in query_ind(row)]+[i for i in query_ind(col)]
    for i in ind_comp_list:
        exempt_set.add(i)
    exempt_set.add(row)
    exempt_set.add(col)
    new_corr=[]
    for iter_company in range(cov_matrix_enlarge.shape[1]):
        if_exempt=iter_company in exempt_set
        if (iter_company>=max_num) and (not if_exempt):
            iter_comp_com=[i for i in query_ind(iter_company)]
            sum_corr=sum([cov_matrix[i,j] for i in iter_comp_com for j in ind_comp_list])/(len(iter_comp_com)*len(ind_comp_list))**1.05
        elif not if_exempt:
            sum_corr=sum([sum_corr_func(i1,iter_company) for i1 in ind_comp_list])/len(ind_comp_list)
        else:
            sum_corr=0
        new_corr.append(sum_corr)
    cov_matrix_enlarge=np.vstack((cov_matrix_enlarge,np.matrix(new_corr)))
    cov_matrix_enlarge=np.hstack((cov_matrix_enlarge,np.transpose(np.matrix(new_corr+[0]))))
    # set the corr within the same industry to 1
    elim_corr(ind_comp_list)
    elim_corr([row,col])
    # record the companies that have been categorized
    for i in ind_comp_list:
        set_num_counter[i]=record_dummy
    record_dummy -= 1

In [31]:
# show the result
ind_dict=dict()
for i in set(set_num_counter):
    ind_dict[i]=set()
for i in range(len(set_num_counter)):
    ind_dict[set_num_counter[i]].add(company_index[i])
for i in ind_dict:
    print(ind_dict[i])

{'600438.SH', '600025.SH', '600674.SH', '601985.SH', '600068.SH', '600023.SH', '600027.SH', '600011.SH', '600795.SH', '600886.SH', '600900.SH'}
{'002422.SZ', '600887.SH', '600519.SH', '600276.SH', '603288.SH', '600809.SH', '002304.SZ', '000858.SZ', '000596.SZ', '000423.SZ', '000568.SZ', '603156.SH'}
{'000001.SZ', '601328.SH', '601998.SH', '601838.SH', '600036.SH', '601169.SH', '600015.SH', '601818.SH', '601398.SH', '600919.SH', '600016.SH', '600000.SH', '601288.SH', '601166.SH', '601229.SH', '600926.SH', '601997.SH', '601009.SH', '600928.SH', '002958.SZ', '601988.SH', '002142.SZ', '601939.SH', '601577.SH'}
{'002475.SZ', '603986.SH', '000413.SZ', '000938.SZ', '000725.SZ', '002916.SZ', '300408.SZ', '603501.SH', '600703.SH', '601012.SH', '002179.SZ', '002938.SZ'}
{'600398.SH', '002027.SZ', '002230.SZ', '002607.SZ', '600588.SH', '601828.SH'}
{'600535.SH', '600085.SH', '600998.SH', '002773.SZ', '000661.SZ', '600606.SH', '002411.SZ', '600436.SH', '000538.SZ'}
{'600221.SH', '600066.SH', '6000

### Official classification

In [15]:
official_class=pd.read_csv("hs300_official_class.csv")
official_class['股票代码']=official_class['股票代码'].map(lambda x:'0'*(6-len(str(x)))+str(x))
official_class.sort_values(by='股票代码',inplace=True)
official_class

,股票代码,主营行业
299,000001,金融
298,000002,房地产
297,000063,信息技术
296,000069,房地产
295,000100,家电
...,...,...
4,603799,有色金属
3,603833,其他
2,603899,轻工制造
1,603986,电子设备


In [17]:
official_class[official_class['主营行业']=='交运设备']
len(official_class.groupby('主营行业').groups)

25

### Comparable Model: k-means

In [32]:
# another comparable model: k-means model
estimator=KMeans(n_clusters=25)
estimator.fit(cov_matrix)
label_pred=estimator.labels_
centroids=estimator.cluster_centers_
inertia=estimator.inertia_

In [43]:
kmeans_dict=dict()
for i in range(25):
    kmeans_dict[i]=set()
for ind in range(max_num):
    kmeans_dict[label_pred[ind]].add(company_index[ind])
for i in kmeans_dict:
    print(kmeans_dict[i])

{'600398.SH', '603993.SH', '002271.SZ', '300498.SZ', '000898.SZ', '601877.SH', '600276.SH', '600547.SH', '002773.SZ', '000568.SZ', '601933.SH', '000786.SZ', '002411.SZ', '002202.SZ', '600585.SH', '002252.SZ', '002010.SZ', '000538.SZ', '002007.SZ'}
{'300347.SZ', '601360.SH', '600196.SH', '002624.SZ', '600118.SH', '601989.SH', '600100.SH', '600867.SH', '000661.SZ', '601919.SH', '300003.SZ', '000338.SZ', '300433.SZ', '300144.SZ', '002607.SZ', '002558.SZ'}
{'600369.SH', '600030.SH', '600999.SH', '002673.SZ', '601066.SH', '601555.SH', '600816.SH', '600837.SH', '601901.SH', '600390.SH', '000776.SZ', '600109.SH', '601788.SH', '000166.SZ', '000783.SZ', '601198.SH', '601377.SH', '601108.SH', '601211.SH', '600958.SH', '601236.SH', '601162.SH', '000728.SZ', '002736.SZ', '601688.SH', '300059.SZ', '601577.SH'}
{'000001.SZ', '601328.SH', '601998.SH', '601838.SH', '600036.SH', '601169.SH', '600015.SH', '601818.SH', '601398.SH', '600919.SH', '600016.SH', '600000.SH', '601288.SH', '601166.SH', '601229.